In [1]:
import sys
sys.path.append("../../")
from data.Frame import Frame
from bk import show_weighted_average, show_group_total, get_pandas_dataframe, line_chart, scatter_plot
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show, reset_output
import sources
from sources.unpd.indicators import get_projection_by_groups as gpp
from sources.cdp.indicators import get_indicators as get_indicators_cdp_2018
from sources.worldbank.datatable import get_data_table as get_wb_tb
reset_output()
output_notebook()

Loading BokehJS ...

# Human and Social development in the LDCs

# The Least Developed Countries

+ The category of the least developed countries was created in 1971

+ Least developed countries (LDCs) are low-income countries confronting severe structural impediments to sustainable development. They are highly vulnerable to economic and environmental shocks and have low levels of human assets.


Least developed countries (LDCs) are **low-income countries** confronting severe **structural** impediments to sustainable development. They are highly **vulnerable to economic and environmental shocks** and have **low levels of human assets**.

## The criteria to identify LDCs
The Committee for Development Policy has established three criteria for including and graduating countries from the category:
    
### The Economic Vulnerability Index (EVI)
    
### The Human Assets Index (HAI)

### GNI per capita

In [2]:
cdp_data_2018 = get_indicators_cdp_2018(['LDCs'])
entities = cdp_data_2018.get_column('entity').as_array()
evi = cdp_data_2018.get_column('evi').as_array()
hai = cdp_data_2018.get_column('hai').as_array()
gni = cdp_data_2018.get_column('gni_per_capita_dollars').as_array()
gni_normalized = [(x / sum(gni))*30 for x in gni]

In [3]:
p = scatter_plot(evi, hai, gni_normalized, entities)
show(p)

# The Istanbul Programme of Action

# Objectives

27. The overarching goal of the Programme of Action for the decade 2011-2020 is to overcome the structural challenges faced by the least developed countries in order to eradicate poverty, achieve internationally agreed development goals and enable graduation from the least developed country category.



28. Guided by the overarching goal, national policies of least developed countries and international support measures during the decade will focus on the following specific objectives with the aim of enabling half the number of least developed countries to meet the criteria for graduation by 2020:

**(a)** Achieve sustained, equitable and inclusive economic growth in least developed countries, to at least the level of 7 per cent per annum, by strengthening their productive capacity in all sectors through structural transformation and overcoming their marginalization through their effective integration into the global economy, including through regional integration;

**(b) Build human capacities by fostering sustained, equitable and inclusive human and social development, gender equality and the empowerment of women;**

**(c)** Reduce the **vulnerability** of least developed countries to economic, natural and environmental shocks and disasters, as well as climate change, and enhance their ability to meet these and other challenges through strengthening their resilience;

**(d)** Ensure enhanced financial resources and their effective use for least developed countries’ development, including through domestic resource mobilization, ODA, external debt relief, foreign direct investment and remittances;

**(e)** Enhance good governance at all levels, by strengthening democratic processes, institutions and the rule of law; increasing efficiency, coherence, transparency and participation; protecting and promoting human rights; and reducing corruption, and strengthen least developed country Governments’ capacity to play an effective role in their economic and social development.

# IV. Priority areas for action

43. The actions will be organized by priority areas as:

A. Productive capacity

B. Agriculture, food security and rural development

C. Trade

D. Commodities

**E. Human and social development**

F. Multiple crises and other emerging challenges ▪Economic shocks

H. Good governance at all levels

# E. Human and social development

+ Education and training

+ Population and primary health

+ Youth development

+ Shelter

+ Water and sanitation

+ Gender equality and empowerment of women 

+ Social protection

# Population and primary health

In [4]:
years = list(map(str, list(range(1962,2018))))
result = show_group_total(years, 'SP.POP.TOTL', ['LDCs'], line_width=3, height=400, width= 1000, marker_width=0)

In [5]:
gr = []
for i in range (0, len(result[0][1])):
    gr.append((result[0][1][i]/result[0][1][i-1])-1)  

In [6]:
print (result[0][0][1] + "-" + result[0][0][55] + " -> " + str(1-(result[0][1][1]/result[0][1][55])))
print (result[0][0][1] + "-" + result[0][0][20] + " -> " + str(1-(result[0][1][1]/result[0][1][20])))
print (result[0][0][20] + "-" + result[0][0][40] + " -> " + str(1-(result[0][1][20]/result[0][1][40])))
print (result[0][0][40] + "-" + result[0][0][5] + " -> " + str(1-(result[0][1][40]/result[0][1][55])))

1963-2017 -> 0.7376189026422728
1963-1982 -> 0.37209015351053276
1982-2002 -> 0.40563803334539317
2002-1967 -> 0.29695319392589503


# Growth rates

1963-2017 -> 73%

1963-1982 -> 37%

1982-2002 -> 40%

2002-2017 -> 29%

In [7]:
# Population growth
result = show_weighted_average(years, 'SP.POP.GROW', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=3)

# Average population growth in the LDCs is almost double than for other developing countries

In [8]:
# GDP growth
years = list(map(str, list(range(1970,2017))))
result = show_weighted_average(years, 'NY.GDP.MKTP.KD.ZG', 'NY.GDP.MKTP.CD', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=2, marker_width = 4)

# Economic and demographic growth are closely intertwined

# Accelerating poverty erradication requires higher growth rates (The IPoA set a target of 7%)

# Economic growth should facilitate the process of economic transformation and support poverty erradication

In [9]:
show(line_chart(gpp(['LDCs']), "Projected population, thousands", dataset='all',
                x_axis_label='Years', y_axis_label='Projected population, thousands', 
                height=400, width=1000, line_width=3))

In [10]:
show(line_chart(sources.unpd.indicators.get_projection_by_groups(['LDCs', 'Developing excluding LDCs'], 
                                                                 dataset='0-24'), "Projected population, thousands", x_axis_label='Years', y_axis_label='Projected population, 0-24, thousands', height=400, width=1000, line_width=4))

In [11]:
show(line_chart(gpp(['LDCs', 'Developing excluding LDCs', 'OECD']), 
                "Projected population, thousands", x_axis_label='Years', 
                y_axis_label='Projected population, thousands', height=400, line_width=3,width=900))

In [12]:
years = list(map(str, list(range(1962,2018))))
# Rural population as a percentage of total population
result = show_weighted_average(years, 'SP.RUR.TOTL.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=3)

In [13]:
years = list(map(str, list(range(2010,2018))))
result = show_weighted_average(years, 'NV.AGR.TOTL.ZS', 'SP.RUR.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=4)

In [14]:
result = show_group_total(years, 'SP.RUR.TOTL', ['LDCs'], 
                                height=400, width=1000, line_width=4)

# Rural job creation is crucial for poverty eradication

 + Smallholder population have an important role in achieving food security

 + Importance of synergies between urban and rural development

 + The majority of the population lives in rural areas in LDCs, yet agriculture generates less than 30% of GDP

In [15]:
years = list(map(str, list(range(2000,2018))))
# Population 0-14
result = show_weighted_average(years, 'SP.POP.0014.TO.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=4)

In [16]:
d = sources.unpd.indicators.get_age_group_by_country_groups("0-29", ['LDCs'])
show(line_chart(d, d[0][3], x_axis_label="Years", y_axis_label='Population below 30', height=400, 
                width=900, line_width=4))

In [17]:
# Prevalence of undernourishment (% of population)
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SN.ITK.DEFC.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=3)

In [18]:
# Food imports as a percentage of merchandise imports
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'TM.VAL.FOOD.ZS.UN', 'TM.VAL.MRCH.CD.WT', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=3)

In [19]:
# Vulnerable employment
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SL.EMP.VULN.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=3)

In [20]:
# Female vulnerable employment
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SL.EMP.VULN.FE.ZS', 'SL.TLF.TOTL.IN', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=3)

In [21]:
# Male vulnerable employment
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SL.EMP.VULN.MA.ZS', 'SL.TLF.TOTL.IN', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=4)

In [22]:
# Life expectancy
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SP.DYN.LE00.IN', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=3)

In [23]:
# Female life expectancy
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SP.DYN.LE00.FE.IN', 'SP.RUR.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=4)

In [24]:
# Maternal mortality ratio
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SH.STA.MMRT', 'SP.POP.TOTL.FE.IN', ['LDCs', 'Developing excluding LDCs'], 
                                height=400, width=1000, line_width=3)

In [25]:
# Infant mortality rate
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SP.DYN.IMRT.IN', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'], 
                                height=400, width=1000, line_width=4)

# Lack of dissagregated gender data is a problem


# Gender responsive development planning


# Great progress of political representation of women in parliament


# Important to promote the representation of women in other domains and across social groups

# Education and training

In [26]:
# School enrollment, primary
years = list(map(str, list(range(2010,2018))))
result = show_weighted_average(years, 'SE.PRM.ENRR', 'SP.POP.TOTL', 
                                ['LDCs', 'Developing excluding LDCs'], 
                                height=400, width=1000, line_width=3)

In [27]:
# School enrollment, secondary
years = list(map(str, list(range(2010,2018))))
result = show_weighted_average(years, 'SE.SEC.ENRR', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs'], 
                                height=400, width=1000, line_width=3)

In [28]:
# School enrollment, tertiary
years = list(map(str, list(range(2010,2015))))
result = show_weighted_average(years, 'SE.TER.ENRR', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'UMICs'], 
                                height=400, width=1000, line_width=3)

In [29]:
# Pupil teacher ratio
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SE.PRM.ENRL.TC.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs'], 
                                height=400, width=1000, line_width=4)

In [30]:
# Trained teachers in primary
years = list(map(str, list(range(2005,2015))))
result = show_weighted_average(years, 'SE.PRM.TCAQ.ZS', 'SE.PRM.TCHR', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=4)

In [31]:
# Trained teachers secondary
years = list(map(str, list(range(2005,2015))))
result = show_weighted_average(years, 'SE.SEC.TCAQ.ZS', 'SE.SEC.TCHR', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=4)

# LDCs have made enormous progress in enrollment at the primary level

# Challenges remain in relation to factors affecting the quality of education

# Gender inequality, specially at secondary and tertiary levels

# Youth development

In [32]:
#Literacy rate, youth, total (% of youth 15-24)
years = list(map(str, list(range(2010,2018))))
result = show_weighted_average(years, 'SE.ADT.1524.LT.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=4)

In [33]:
# Literacy rate, youth, total (% of youth 15-24)
years = list(map(str, list(range(2011,2018))))
result = show_weighted_average(years, 'SL.UEM.NEET.ZS', 'SP.POP.TOTL', 
                               ['LDCs'], 
                                height=400, width=1000, line_width=4)

# Support training to provide quality education


# Unemployment rate is substantially higher for youth population


# Underemployment and vulnerable employment particularly affect the youth and in many countries has a gender dimension


# Promote employment opportunities for the youth

# Shelter

In [34]:
# Percentage of urban population living in slums
years = ['2000', '2005', '2007', '2009', '2014']
result = show_weighted_average(years, 'EN.POP.SLUM.UR.ZS', 'SP.URB.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=4)

In [35]:
years = list(map(str, list(range(1960,2018))))
# Urban population as a percentage of total population
result = show_weighted_average(years, 'SP.URB.TOTL.IN.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=4)

In [36]:
result = show_group_total(years, 'SP.URB.TOTL', ['LDCs'], 
                                height=400, width=1000, line_width=4)

In [37]:
# Growth of urban population
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SP.URB.GROW', 'SP.URB.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=4)

# Urban areas are rapidly increasing in LDCs

# Two thirds of the urban space in the LDCs will have to be built in the next 35 years

# The rate of urbanization in LDCs is very high. Cities will have to absorb more people in less time

# Adressing the provision of services in a context of low resources and lack of technical capacities constitute a huge challenge

# Water and sanitation

In [38]:
# Access to safely managed sanitation services
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SH.STA.SMSS.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=4)

In [39]:
# Basic water access
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SH.H2O.BASW.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'], 
                                height=400, width=1000, line_width=4)

In [40]:
# Basic water access rural
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SH.H2O.BASW.RU.ZS', 'SP.RUR.TOTL', ['LDCs', 'Developing excluding LDCs'], 
                                height=400, width=1000, line_width=4)

# Gender equality and the empowerment of women

In [41]:
# Family planning
years = ['2004', '2007', '2009', '2014', '2016']
result = show_weighted_average(years, 'SH.FPL.SATM.ZS', 'SP.POP.TOTL.FE.IN', 
                               ['LDCs', 'Developing excluding LDCs'], 
                                height=400, width=1000, line_width=4)

In [42]:
# Basic water access rural
years = ['2000', '2003', '2006', '2009', '2014', '2017']
result = show_weighted_average(years, 'SP.ADO.TFRT', 'SP.POP.TOTL.FE.IN', 
                               ['LDCs', 'Developing excluding LDCs'], 
                                height=400, width=1000, line_width=4)

# Social protection

In [43]:
# Age dependency ratio (% of working-age population)
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SP.POP.DPND', 'SP.POP.1564.TO', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=4)

In [44]:
#Employment to population ratio, 15+, total (%) (modeled ILO estimate)
# Age dependency ratio (% of working-age population)
years = list(map(str, list(range(2000,2018))))
result = show_weighted_average(years, 'SL.EMP.TOTL.SP.ZS', 'SP.POP.TOTL', 
                               ['LDCs', 'Developing excluding LDCs', 'OECD'], 
                                height=400, width=1000, line_width=3)

# Some very general ideas

# The world is becoming increasingly unequal

# Great progress in certain areas, still an awful lot to do

# Sustainable development in the LDCs won't happen without addressing gender inequalities

# The SDGs and other internationally agreed development goals won't happen without accelerated progress in the LDCs.

# Many thanks

Let's stay in touch

Tomás González
gonzalezt@un.org
